# Batch images processing

## Inputs

In [1]:
# Import libraries
from calibrations import calibrate_color, build_calibration, calibrate_distortion, calibrate_color_and_distortion
import os
from plantcv import plantcv as pcv
import shutil
import pandas as pd
from aux_functions import slicing, obtain_pixel_metric, divide_in_sets, ungroup_pic
from model_class import model_segmentation
from pictures_class import pictures


In [2]:
#Inputs
working_directory="C:/Users/Pheno/Documents/database_almondcv2/"
raw_folder="C:/Users/Pheno/Documents/database_almondcv2/all_sessions/"
chessboards="C:/Users/Pheno/Documents/database_almondcv2/calibracion/chessboards"
mtx_input_path=os.path.join(chessboards,"calibration_mtx.npz") ## for distorsion
standard_matrix_color="C:/Users/Pheno/Documents/database_almondcv2/all_sessions/02_11_F-010.JPG" ### a picture to select if some pictures produces errors in color 
info_data="C:/Users/Pheno/Documents/database_almondcv2/info_data.txt"
models_directory="C:/Users/Pheno/Documents/database_almondcv2/models/"
pre_model=os.path.join(models_directory, "yolov8n-seg.pt")
group_model_path=os.path.join(models_directory, "rectangles_yolo_11s_04102024.pt")
shell_model_path=os.path.join(models_directory, "shell_320slides_imgsz_640_yolov8n.pt")
zip_file_coin=os.path.join(working_directory,"coin_seg_640_04102024.zip")
zip_file_group=os.path.join(working_directory,"rectangles_seg_04102024.zip")
zip_file_shell=os.path.join(working_directory,"slices_mixpx_12102024.zip")
#outputs
output="C:/Users/Pheno/Documents/database_almondcv2"
output_calibrated=os.path.join(output,"calibrated_pics/")
os.makedirs(output_calibrated, exist_ok=True)

## Color and distorsion correction

In [3]:
# If you need to create distortion model
build_calibration(chessboardSize=(6, 8), frameSize=(5472,3648), dir_path=chessboards, 
                  image_format=".jpg", size_of_chessboard_squares_mm=20)

In [3]:
#Ahora mismo esta puesto en modo calibracion los errores utilizando una foto como referencia, para calibracion normal quitar la standard_matrix_color
calibrate_color_and_distortion(raw_folder=raw_folder,mtx_input_path=mtx_input_path,output_calibrated=output_calibrated, radius_param=10, standard_matrix=standard_matrix_color)

In [ ]:
#CHUNK para mover automaticamente lo que este en la lista errores a donde tu queiras
#first we move 

carpeta_destino = os.path.join(raw_folder,"errores")    # Cambia esto por la ruta correcta

# Crea la carpeta de destino si no existe
os.makedirs(carpeta_destino, exist_ok=True)


# Lee el archivo de texto con los nombres de las imágenes
with open(os.path.join(output_calibrated,"errors_in_calibrations.txt"), "r") as archivo:
    for nombre_imagen in archivo:
        nombre_imagen = nombre_imagen.strip()  # Elimina espacios en blanco y saltos de línea
        ruta_imagen_origen = os.path.join(raw_folder, nombre_imagen)
        
        # Verifica si el archivo existe antes de moverlo
        if os.path.exists(ruta_imagen_origen):
            # Mueve la imagen a la carpeta de destino
            shutil.copy(ruta_imagen_origen, carpeta_destino)
            print(f"Moviendo: {nombre_imagen} a {carpeta_destino}")
        else:
            print(f"No se encontró la imagen: {nombre_imagen}")

## Load info data

In [ ]:
#Cargar info_data, se puede cargar ya una columna pixelmetric con la informacion pixel-mm o obtener la metrica pixel desde las imagenes.
info_data_df=pd.read_csv(info_data,sep="\t")
info_data_df

### Obtain pixel_metric from the pictures

In [ ]:
# Si no tienes un modelo para reconocer tu objeto de referencia debes crearlo
#Se comienza haciendo particiones de la imagen con la función slicing para introducirlas en CVAT

slicing(input_folder=output_calibrated,output_directory=working_directory,name_slicing="Slices_for_coin_03102024", number_pictures=60, crop="left", slice_height=640, slice_width=640)


In [ ]:
# Posteriormente se tiene que segmentar en CVAT, y ese output se leera para entrenar el modelo y ver si funciona bien, para ello utilizamos la función
# train model, pon el archivo.zip en el working directory.

coin_model=model_segmentation(working_directory=working_directory)
coin_model.train_segmentation_model(input_zip=zip_file_coin, epochs=80,imgsz=640, name_segmentation="coin_640", pre_model=pre_model)

In [ ]:
# Si tenemos el modelo ya simplemente podemos cargarlo y aplicarlo sobre la carpeta que queramos
model_path=os.path.join(models_directory,"coin_yolo_11s_041022024.pt")
coin_model_saved_sahi=model_segmentation(working_directory=working_directory)
contours_coin=coin_model_saved_sahi.predict_model_sahi(model_path=model_path,
                                                       folder_input="C:/Users/Pheno/Documents/database_almondcv2/calibrated_pics/",
                                                       slice_height=640, slice_width=640, check_result=True)


In [20]:
info_data_completed=obtain_pixel_metric(info_data=info_data_df, contours=contours_coin,
                                         output_directory=working_directory, reference=24.25)

### Desagrupar imagenes

In [ ]:
# Una vez hemos completado ya el pixelmetric, si tenemos fotos agrupadas, debemos desagruparlas y conectarlas con su ID
#Para posteriormente sacar las medidas que queramos. Esta vez no hacemos slicing, peusto que los grupos son mas grandes que las slcies.
divide_in_sets(input_folder=output_calibrated,output_directory=working_directory, division_name="rectangle_group", number_pictures=60)



In [ ]:
#Ahora vamos a entrenar un modelo para que reconozca los grupos

group_model=model_segmentation(working_directory=working_directory)
group_model.train_segmentation_model(input_zip=zip_file_group, epochs=100,imgsz=1280,
                                      name_segmentation="rectangle_group", pre_model=pre_model, batch=8)

In [ ]:
#ESTO DESPUES PARA LOS CONTORNOS EN GROUPED PICTURES

# Si tenemos el modelo ya simplemente podemos cargarlo y aplicarlo sobre la carpeta que queramos

group_model_saved=model_segmentation(working_directory=working_directory)
contours_groups=group_model_saved.predict_model(model_path=group_model_path,
                               folder_input=output_calibrated,
                               imgsz=1280, check_result=True)

In [ ]:
#Ahora que ya tenemos los contornos, podemos aplicarlo a nuestras imagenes, y tambien coordinaar el info_data_df
# info_data_completed_path=os.path.join(working_directory, "info_data_completed.txt")
# info_data_completed=pd.read_csv(info_data_completed_path,sep="\t")
ungroup_pic(input_contours=contours_groups, output_path=working_directory, info_file=info_data_completed)

## Measuring

### Train a model

In [ ]:
#Vamos a entrenar un modelo para las almendras con cascara
#Vamos a separar en una carpeta las imagenes de almendras con cascara
info_data_df=pd.read_csv(f"{working_directory}/info_data_completed_ungrouped.txt", sep="\t")
info_data_df = info_data_df[info_data_df["Shell"] == "Yes"]

# Directorio donde están las imágenes
directorio_imagenes = f"{working_directory}/Ungrouped_pics"  # Cambia esto por la ruta correcta
# Directorio de destino donde copiarás las imágenes
directorio_destino = f"{working_directory}/Shell_ungrouped_pics"

# Crear el directorio de destino si no existe
os.makedirs(directorio_destino, exist_ok=True)

# Copiar las imágenes
for index, row in info_data_df.iterrows():
    imagen_relativa = row['Sample_picture']  # Obtener la ruta relativa de la imagen
    imagen_path = os.path.join(directorio_imagenes, imagen_relativa)  # Ruta completa

    try:
        # Copiar la imagen al directorio de destino
        shutil.copy(imagen_path, os.path.join(directorio_destino, os.path.basename(imagen_path)))
        print(f"Imagen copiada: {imagen_path} a {directorio_destino}")
    except Exception as e:
        print(f"No se pudo copiar la imagen {imagen_path}. Error: {e}")

In [ ]:
#Bien ahora vamos a hacer el slicing y a segmentar y etiquetar las imagenes

slicing(input_folder=directorio_destino,output_directory=working_directory,name_slicing="Slices_for_shell_10102024_320", number_pictures=25, slice_height=320, slice_width=320)

In [ ]:
#Ahora vamos a entrenar el modelo 

shell_model=model_segmentation(working_directory=working_directory)
shell_model.train_segmentation_model(input_zip=zip_file_shell, epochs=50,imgsz=640, name_segmentation="shell_pxmx_121024",
                                      pre_model=pre_model, batch=16)


In [5]:
#Ahora podemos utilizar ya el modelo y la clase picture para obtener todas las medidas

info_data_df=pd.read_csv(f"{working_directory}/info_data_completed_ungrouped.txt", sep="\t")
info_data_df = info_data_df[info_data_df["Shell"] == "Yes"]
input_folder=os.path.join(working_directory, "fotos_prueba")
shell_masks=model_segmentation(working_directory=working_directory)
shell_masks=shell_masks.slice_predict_reconstruct(input_folder=input_folder,imgsz=640, model_path=shell_model_path,
                                          slice_height=320, slice_width=320,overlap_height_ratio=0.2,
                                          overlap_width_ratio=0.2)
shell_object=pictures(working_directory=working_directory, input_folder=input_folder,info_file=info_data_df,
                      fruit="Shell_almond", binary_masks=True, project_name="Shell_2022_13102024_smooth_5, watershed",
                        segmentation_maks=shell_masks, smoothing=True, smoothing_iterations=2, kernel_smoothing=5,
                        watershed=True, kernel_watershed=5, threshold_watershed=0.5)
shell_object.measure_almonds()

10/14/2024 11:35:54 - INFO - sahi.slicing -   image.shape: (757, 2901)
10/14/2024 11:35:54 - INFO - sahi.slicing -   Num slices: 36 slice_height: 320 slice_width: 320


GPU detectada: NVIDIA GeForce RTX 3060
Memoria total de la GPU: 12.00 GB


10/14/2024 11:35:58 - INFO - sahi.slicing -   image.shape: (757, 2985)
10/14/2024 11:35:58 - INFO - sahi.slicing -   Num slices: 36 slice_height: 320 slice_width: 320
10/14/2024 11:36:03 - INFO - sahi.slicing -   image.shape: (757, 2968)
10/14/2024 11:36:03 - INFO - sahi.slicing -   Num slices: 36 slice_height: 320 slice_width: 320
10/14/2024 11:36:07 - INFO - sahi.slicing -   image.shape: (774, 2951)
10/14/2024 11:36:07 - INFO - sahi.slicing -   Num slices: 36 slice_height: 320 slice_width: 320
10/14/2024 11:36:11 - INFO - sahi.slicing -   image.shape: (757, 2951)
10/14/2024 11:36:11 - INFO - sahi.slicing -   Num slices: 36 slice_height: 320 slice_width: 320
c:\Users\Pheno\OneDrive - UNIVERSIDAD DE MURCIA\Escritorio\Almond_CV\almond_cv2\pictures_class.py:292: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old beh

## SUCIO

In [ ]:
input_folder=os.path.join(working_directory, "fotos_prueba")
prueba1=model_segmentation(working_directory=working_directory)
prueba2=prueba1.slice_predict_reconstruct(input_folder=input_folder,imgsz=640, model_path=shell_model_path,
                                          slice_height=320, slice_width=320,overlap_height_ratio=0.2,
                                          overlap_width_ratio=0.2)


In [ ]:
prueba2[200][0]

In [ ]:
from PIL import Image
from sahi.slicing import slice_image
image_path=standard_matrix_color
image_selected = Image.open(image_path)
image_sliced=slice_image(image=image_selected, slice_width=640,
                        slice_height=640,overlap_height_ratio=0.2,
                        overlap_width_ratio=0.2, verbose=True)

In [ ]:
image_sliced.original_image_height